In [ ]:
import os
from pathlib import Path
from typing import Annotated

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import seaborn as sns
import tifffile

from sklearn.cluster import KMeans
from skimage.color import label2rgb
from sklearn.neighbors import radius_neighbors_graph
from sklearn.neighbors import NearestNeighbors

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests
from scipy.stats import entropy, chi2_contingency
from matplotlib.backends.backend_pdf import PdfPages
from statannotations.Annotator import Annotator

from scipy import sparse

plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42 #make text editable in pdf

os.chdir('/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/Xenium/analysis/lineage_specific_analysis/MSC/')
os.getcwd()


In [ ]:
merged = sc.read_h5ad("/diskmnt/Projects/myeloma_scRNA_analysis/MMY_IRD/Xenium/analysis/radial_neighborhoods/Output/merged_RN.h5ad")
# make a new row combining UPN and collection
merged.obs['UPN_Collection'] = merged.obs['UPN'].astype('string').str.cat(merged.obs['Collection'].astype('string'), sep='_', na_rep='')

sample_to_collection = merged.obs.set_index('Sample')['Collection'].to_dict()
uc_to_collection = merged.obs.set_index('UPN_Collection')['Collection'].to_dict()
uc_to_upn =  merged.obs.set_index('UPN_Collection')['UPN'].to_dict()
collection_order = ["NBM", "NDMM", "PT"] 
timecols = {"NBM": "#0C7515", "NDMM": "#E619B9", "PT": "#CF99C3"} 

strom = merged[merged.obs['ct'].isin(['MSC', 'Endothelial', 'Adipocyte', 'vSMC/Pericyte', 'Fibro/Osteo'])].copy()
strom.obs['ct'].unique().tolist()

In [ ]:
sc.pl.matrixplot(
            strom[strom.obs['ct'] == 'MSC'].copy(),
            var_names=genes,
            groupby='Collection',
            standard_scale='var',
            dendrogram=False,
            show=False 
        )
plt.savefig("strom_sen_matrixplot_MSC.pdf")

In [ ]:
def remove_outliers_iqr(df, col='frac_ct', k=1.5):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - k * iqr
    upper = q3 + k * iqr
    return df[(df[col] >= lower) & (df[col] <= upper)]

def compute_positive_fraction(adata, gene, threshold=0):
    x = adata[:, gene].X
    x = x.toarray().ravel()
    pos = x > threshold
    
    needed_cols = ["Collection", "UPN"]
    obs_cols = [c for c in needed_cols if c in adata.obs.columns]
    df = adata.obs[obs_cols].copy()
    df["pos"] = pos
    group_cols = [col for col in ["UPN", "Collection"] if col in df.columns]

    out = (
        df.groupby(group_cols, observed=True)
          .agg(frac_pos=("pos", "mean"), n_cells=("pos", "size"))
          .reset_index()
    )
    out["gene"] = gene
    return out


In [ ]:
collection_order = ["NBM", "NDMM", "PT"]     
pdf_path = "gene_positive_fraction_MSC_boxplots.pdf"
threshold = 0     

all_results = []
adata=strom[strom.obs['ct']=='MSC'].copy()
#adata=strom.copy()

for gene in genes:
    df_gene = compute_positive_fraction(adata, gene, threshold=threshold)
    all_results.append(df_gene)

df_all = pd.concat(all_results, ignore_index=True)

with PdfPages(pdf_path) as pdf:
    for gene in genes:
        sub = df_all[df_all["gene"] == gene].copy()

        plt.figure(figsize=(2, 4))
        ax = sns.boxplot(
            data=remove_outliers_iqr(sub,col='frac_pos'),
            x="Collection",
            y="frac_pos", palette=timecols,
            order=collection_order,
            fliersize=0
        )
        sns.stripplot(
            data=remove_outliers_iqr(sub,col='frac_pos'),
            x="Collection",
            y="frac_pos",
            order=collection_order,
            color="black",
            size=3,
            alpha=0.8,
            jitter=True
        )
        pairs = []
        # generate all ordered pairs in collection_order
        for i in range(len(collection_order)):
            for j in range(i+1, len(collection_order)):
                pairs.append((collection_order[i], collection_order[j]))

        annot = Annotator(
            ax, pairs, data=sub,
            x="Collection", y="frac_pos",
            order=collection_order
        )
        annot.configure(
            test="Mann-Whitney",
            text_format="star",
            comparisons_correction='BH',
            hide_non_significant=True,
            loc="inside",
            line_height=0.03,
            line_offset=0.02
        )
        annot.apply_and_annotate()

        plt.title(f"{gene}")
        plt.ylabel("% Positive")
        plt.xlabel("Collection")
        plt.tight_layout()
        pdf.savefig()
        plt.close()

In [ ]:
collection_order = ["NBM", "NDMM", "PT"]     
pdf_path = "gene_positive_fraction_fibost_boxplots.pdf"
threshold = 0     

all_results = []
adata=strom[strom.obs['ct']=='Fibro/Osteo'].copy()
#adata=strom.copy()

for gene in genes:
    df_gene = compute_positive_fraction(adata, gene, threshold=threshold)
    all_results.append(df_gene)

df_all = pd.concat(all_results, ignore_index=True)

with PdfPages(pdf_path) as pdf:
    for gene in genes:
        sub = df_all[df_all["gene"] == gene].copy()

        plt.figure(figsize=(2, 4))
        ax = sns.boxplot(
            data=remove_outliers_iqr(sub,col='frac_pos'),
            x="Collection",
            y="frac_pos", palette=timecols,
            order=collection_order,
            fliersize=0
        )
        sns.stripplot(
            data=remove_outliers_iqr(sub,col='frac_pos'),
            x="Collection",
            y="frac_pos",
            order=collection_order,
            color="black",
            size=3,
            alpha=0.8,
            jitter=True
        )
        pairs = []
        # generate all ordered pairs in collection_order
        for i in range(len(collection_order)):
            for j in range(i+1, len(collection_order)):
                pairs.append((collection_order[i], collection_order[j]))

        annot = Annotator(
            ax, pairs, data=sub,
            x="Collection", y="frac_pos",
            order=collection_order
        )
        annot.configure(
            test="Mann-Whitney",
            text_format="star",
            comparisons_correction='BH',
            hide_non_significant=True,
            loc="inside",
            line_height=0.03,
            line_offset=0.02
        )
        annot.apply_and_annotate()

        plt.title(f"{gene}")
        plt.ylabel("% Positive")
        plt.xlabel("Collection")
        plt.tight_layout()
        pdf.savefig()
        plt.close()

In [ ]:
df_all[df_all['Collection']=='PT']

In [ ]:
genes = ["CDKN1A", "SERPINE1", "BMP4", "DKK1"]  
collection_order = ["NBM", "NDMM", "PT"]     
pdf_path = "gene_positive_fraction_MSC_panel_boxplots_pairedWilcoxon.pdf"
threshold = 0     

adata = strom[strom.obs['ct']=='MSC'].copy()

all_results = []
for gene in genes:
    df_gene = compute_positive_fraction(adata, gene, threshold=threshold)
    all_results.append(df_gene)

df_all = pd.concat(all_results, ignore_index=True)

# get paired upns
tmp = (
    df_all[["UPN", "Collection"]]
      .drop_duplicates()
      .query("Collection in ['NDMM', 'PT']")
)

paired_upns = (
    tmp.groupby("UPN")["Collection"]
       .nunique()
)
paired_upns = paired_upns[paired_upns == 2].index.tolist()

paired_upns
upn_cols= {'WU007':"lightgray", 'WU025':"lightgray", 'WU030':"lightgray", 'WU043':"black", 'WU050':"lightgray", 'WU066':"lightgray", 'WU068':"lightgray", 'WU107':"lightgray"}



with PdfPages(pdf_path) as pdf:
    for gene in genes:
        sub = df_all[
            (df_all["gene"] == gene) &
            (df_all["UPN"].isin(paired_upns))
        ].copy()

        sub_no_out = remove_outliers_iqr(sub, col='frac_pos')

        # sort by UPN then Collection so Wilcoxon pairs align by UPN
        sub_no_out = sub_no_out.sort_values(["UPN", "Collection"])

        plt.figure(figsize=(2.2, 4))
        ax = sns.boxplot(
            data=sub_no_out,
            x="Collection",
            y="frac_pos",
            palette=timecols,
            fliersize=0
        )
        sns.stripplot(
            data=sub_no_out,
            x="Collection",
            y="frac_pos",
            color="black",
            size=3,
            alpha=0.8,
            jitter=True
        )
        # paired lines
        for upn, g in sub_no_out.groupby("UPN"):
            g_ndmm = g[g["Collection"] == "NDMM"]
            g_pt   = g[g["Collection"] == "PT"]
            if len(g_ndmm) == 1 and len(g_pt) == 1:
                y_vals = [g_ndmm["frac_pos"].iloc[0], g_pt["frac_pos"].iloc[0]]
                ax.plot([1, 2], y_vals, color=upn_cols[upn],linewidth=1, alpha=0.7)

        pairs = [("NDMM", "PT")]
        annot = Annotator(
            ax,
            pairs,
            data=sub,
            x="Collection",
            y="frac_pos",
        )
        annot.configure(
            test="Wilcoxon",      
            text_format="star",
            comparisons_correction=None,
            hide_non_significant=True,
            loc="inside",
            line_height=0.03,
            line_offset=0.02,
        )
        annot.apply_and_annotate()

        plt.title(f"{gene} (paired NDMM vs PT)")
        plt.ylabel("% Positive")
        plt.xlabel("Collection")
        plt.tight_layout()
        pdf.savefig()
        plt.close()